In [ ]:
import pandas as pd
import numpy as np
from volcenginesdkarkruntime import Ark

def get_embedding(question, dim=512):
    client = Ark(
        base_url="https://ark.cn-beijing.volces.com/api/v3",
        api_key="44ea595c-61c3-4af1-87fb-50464b5cc95a",
    )
    resp = client.embeddings.create(
        model="ep-20250418144025-nmvsv",
        input=question
    )
    return resp.data[0].embedding

def sliced_norm_l2(vec, dim=2048):
    import numpy as np
    vec = np.array(vec[:dim], dtype=np.float32)
    norm = np.linalg.norm(vec).astype(np.float32)
    return vec / norm

def retrival_top_k(__df__, query_vector, k=30):
    import numpy as np
    import pandas as pd
    """
    input: datafram with "id_", "embedding"
    output: DataFrame with "id_", "dot_product(score)"
    """
    embeddings_matrix = np.vstack(__df__['embedding'].values)
    dot_products = np.dot(embeddings_matrix, query_vector)
    top_k_indices = np.argpartition(dot_products, -k)[-k:]
    top_k_indices = top_k_indices[np.argsort(-dot_products[top_k_indices])]
    result_df = __df__.iloc[top_k_indices].copy()[["id_"]]
    result_df['dot_product'] = dot_products[top_k_indices]
    return result_df


df = pd.read_parquet("data_TA/A_abstract_emb.parquet")
df_paper = pd.read_csv("all_TA_meta.csv")
df['embedding'] = df['embedding'].apply(sliced_norm_l2)


query_vector = sliced_norm_l2(get_embedding("紫杉醇的靶点是什么？"))
retrival_df = retrival_top_k(df, query_vector, k=500)
for _, row in retrival_df[-10:].iterrows():
    paper_id = row["id_"]
    title = df_paper[df_paper["T_id"]==paper_id]["title"].values[0]
    abstract = df_paper[df_paper["T_id"]==paper_id]["abstract"].values[0]
    print(title)
    print(abstract)
    print()

